In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
df = pd.read_csv("OASIS4_data_clinical.csv")

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt

# Your plotting code here

plt.show()  # Display the plot in the Jupyter Notebook output


In [4]:
# Import the necessary module
from ydata_profiling import ProfileReport

# Create a ProfileReport object
report = ProfileReport(df)

# Generate the data profiling report
report.to_file('oasis_4_2.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df.dropna(inplace=True)

In [6]:
df.drop('oasis_id' , axis=1,inplace=True)
df.drop('demographics_id', axis=1,inplace=True)
df.drop('demographics_firstvisit', axis=1,inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 662
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sex                   660 non-null    int64  
 1   age                   660 non-null    int64  
 2   edu                   660 non-null    int64  
 3   race                  660 non-null    int64  
 4   hispanic              660 non-null    int64  
 5   marriage              660 non-null    int64  
 6   declong               660 non-null    float64
 7   decage                660 non-null    int64  
 8   smoke                 660 non-null    int64  
 9   height                660 non-null    float64
 10  weight                660 non-null    float64
 11  bmi                   660 non-null    float64
 12  health_history1       660 non-null    int64  
 13  health_history2       660 non-null    int64  
 14  health_history3       660 non-null    int64  
 15  health_history4       6

In [8]:
# check all the columns in df and convert all the int columns to float, except 'cdr'
for col in df.columns:
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64')

##### Train-Test Split

In [9]:
df.drop('hispanic', axis=1,inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 662
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sex                   660 non-null    float64
 1   age                   660 non-null    float64
 2   edu                   660 non-null    float64
 3   race                  660 non-null    float64
 4   marriage              660 non-null    float64
 5   declong               660 non-null    float64
 6   decage                660 non-null    float64
 7   smoke                 660 non-null    float64
 8   height                660 non-null    float64
 9   weight                660 non-null    float64
 10  bmi                   660 non-null    float64
 11  health_history1       660 non-null    float64
 12  health_history2       660 non-null    float64
 13  health_history3       660 non-null    float64
 14  health_history4       660 non-null    float64
 15  health_history5       6

In [11]:
# encode final_dx
df.drop('final_dx', axis=1,inplace=True)

In [12]:
from sklearn.model_selection import train_test_split
x = df.drop(['final_dx_categorized'], axis=1)
y = df[['final_dx_categorized']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
columns_to_smote = x_train.columns[12:22]  # Assuming 12 to 21 are the column indices
x_train_resampled, y_train_resampled = sm.fit_resample(x_train[columns_to_smote], y_train)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_subset = x_train.iloc[:, 2:]
x_train_scaled = scaler.fit_transform(x_train_subset)
x_train.iloc[:, 2:] = x_train_scaled

x_test_subset = x_test.iloc[:, 2:]
x_test_scaled = scaler.transform(x_test_subset)
x_test.iloc[:, 2:] = x_test_scaled


In [ ]:
import csv
import os
class CSVLogger:
    def __init__(self, filename, fieldnames):
        self.filename = filename
        self.fieldnames = fieldnames

        # Create and initialize the CSV file
        if not os.path.isfile(self.filename):
            with open(self.filename, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
                writer.writeheader()
        else:
            with open(self.filename, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
                writer.writerow({})

    def log(self, data):
        # Append data to the CSV file
        with open(self.filename, 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
            writer.writerow(data)

In [ ]:
# create a CSVLogger object
csv_logger = CSVLogger( 'results.csv', ['model', 'accuracy'] )
